In [ ]:
#Model-2 - Ridge Regression-
# Load the dataset
path= "/content/sample_data/Anonymized Dataset for Info Science MS.xlsx"
df = pd.read_excel(path)

#Lets imput missing values
df['TermGPA'] = df['TermGPA'].fillna(df['TermGPA'].mean())
df['CumulativeGPA'] = df['CumulativeGPA'].fillna(df['CumulativeGPA'].mean())

#Drop insignificant column "FakeIdentifier"
df.drop('FakeIdentifier', axis=1, inplace=True)

#Drop the blank rows input
df = df[~((df['UnitsPassednotincludedinGPA'] == 0) &
          (df['NumberofClassesEnrolled'] == 0))]

# One Hot Encode nominal predictors
nominal_cols = ['Gender', 'PrimaryMilitaryAffiliation', 'College',
                'UAFullTimePartTime', 'FirstGenerationFlag', "AcademicCareer"]
df = pd.get_dummies(df, columns=nominal_cols, drop_first=True)

#Convert boolean columns → int
bool_cols = df.select_dtypes(include="bool").columns
for col in bool_cols:
    df[col] = df[col].astype(int)

# Ordinal Encoding
df["AcademicLevelEndofTerm"]=df["AcademicLevelEndofTerm"].replace(
    {"Freshman": 1, "Sophomore": 2, "Junior":3, "Senior":4, "Graduate":5, "Masters":6 }
)
df["AcademicYear"]=df["AcademicYear"].replace(
    {"2020":1, 2021:2, 2022:3, 2023:4, 2024:5, 2025:6}
)

# Drop highly correlated predictors
cols = ['College_James E Rogers College of Law', 'UnitsPassedincludedinGPA',
        "AcademicCareer_Undergraduate","UAFullTimePartTime_P"]
df.drop(columns=cols, inplace=True)

# ----------------------
# Standardize numeric predictors
# ----------------------
from sklearn.preprocessing import StandardScaler

numeric_columns = ["Age","NumberofClassesEnrolled","UnitsPassednotincludedinGPA",
                    "TermGPA","CumulativeGPA","AcademicLevelEndofTerm","AcademicYear"]
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Prepare X and y
X = df.drop(columns=['CumulativeGPA']).copy()
y = df['CumulativeGPA']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# -----------------------------
# Ridge Regression
# -----------------------------
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)

# Predictions
y_train_pred = ridge.predict(X_train)
y_test_pred = ridge.predict(X_test)

# Performance metrics
train_metrics = pd.DataFrame({
    "Metric": ["R2", "MAE", "MSE", "RMSE"],
    "Value": [
        r2_score(y_train, y_train_pred),
        mean_absolute_error(y_train, y_train_pred),
        mean_squared_error(y_train, y_train_pred),
        np.sqrt(mean_squared_error(y_train, y_train_pred))
    ]
})

test_metrics = pd.DataFrame({
    "Metric": ["R2", "MAE", "MSE", "RMSE"],
    "Value": [
        r2_score(y_test, y_test_pred),
        mean_absolute_error(y_test, y_test_pred),
        mean_squared_error(y_test, y_test_pred),
        np.sqrt(mean_squared_error(y_test, y_test_pred))
    ]
})

# Ridge Coefficient Table
ridge_coef_table = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": ridge.coef_
})


# Get p-values and Confidence Intervals using Statsmodels OLS


import statsmodels.api as sm

X_ols = sm.add_constant(X_train)  # intercept added
ols_model = sm.OLS(y_train, X_ols).fit()

# Extract coefficients significance
ols_table = pd.DataFrame({
    "Feature": ["Intercept"] + list(X.columns),
    "Coefficient": ols_model.params.values,
    "Std Error": ols_model.bse.values,
    "t-Value": ols_model.tvalues.values,
    "p-Value": ols_model.pvalues.values,
    "Lower 95% CI": ols_model.conf_int()[0].values,
    "Upper 95% CI": ols_model.conf_int()[1].values
})

# Generate Ridge Regression Model Equation
ridge_intercept = ridge.intercept_
ridge_coeffs = ridge.coef_

equation_text = "CumulativeGPA = {:.4f}".format(ridge_intercept)

for feature, coef in zip(X.columns, ridge_coeffs):
    sign = " + " if coef >= 0 else " - "
    equation_text += f"{sign}{abs(coef):.4f} * {feature}"

# RIDGE REGRESSION EQUATION
print("\n=================== RIDGE REGRESSION EQUATION ===================")
print(equation_text)
equation_df = pd.DataFrame({"Model_Equation": [equation_text]})
# Save Everything to Excel


output_path = "/content/Model2_Ridge_Results.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    train_metrics.to_excel(writer, sheet_name="Train_Metrics", index=False)
    test_metrics.to_excel(writer, sheet_name="Test_Metrics", index=False)
    ridge_coef_table.to_excel(writer, sheet_name="Ridge_Coefficients", index=False)
    ols_table.to_excel(writer, sheet_name="OLS_pvalues", index=False)
    equation_df.to_excel(writer, sheet_name="Model_Equation", index=False)
    pd.DataFrame({"y_test": y_test, "y_test_pred": y_test_pred}).to_excel(
        writer, sheet_name="Predictions", index=False
    )

print(f"\nExcel file saved at: {output_path}")
